In [17]:
import os
from msgraph.generated.models.o_data_errors.o_data_error import ODataError
from msgraph import GraphServiceClient
from msgraph.generated.users.users_request_builder import UsersRequestBuilder
from azure.identity.aio import ClientSecretCredential
import json
import urllib.request
import urllib.parse
import pandas as pd
import re


In [38]:
def get_secret(secret_name):
    import os
    return os.environ.get(secret_name)

appId = get_secret('GRAPH_CLIENT_ID')
appSecret = get_secret('GRAPH_CLIENT_SECRET') 
tenantId = get_secret('GRAPH_CLIENT_TENANT')
print("If your secrets were initialized properly you should see your tenant id.\nTenant Id: %s " % tenantId)

If your secrets were initialized properly you should see your tenant id.
Tenant Id: c5263ca1-a2e1-42fd-81bb-effcd1666efd 


In [39]:
client_credential = ClientSecretCredential( tenantId, appId, appSecret )
scopes = ['https://graph.microsoft.com/.default']
client = GraphServiceClient(credentials=client_credential, scopes=scopes) 


In [40]:
query_params = UsersRequestBuilder.UsersRequestBuilderGetQueryParameters(
    # Only request specific properties
    select = ['displayName', 'id', 'mail'],
    # Get at most 25 results
    #top = 2,
    # Sort by display name
    orderby= ['displayName']
)

request_config = UsersRequestBuilder.UsersRequestBuilderGetRequestConfiguration(
    query_parameters=query_params
)

userId,displayName,emailAddress = [],[],[]

user_response = await client.users.get(request_configuration=request_config)

for user in user_response.value:
 userId.append(user.id)
 displayName.append(user.display_name)
 emailAddress.append(user.user_principal_name)
 
users_dict = {'userId':userId,'displayName':displayName,'emailAddress':emailAddress}
users_df = pd.DataFrame(data=users_dict) 
users_df.info()
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   userId        100 non-null    object
 1   displayName   100 non-null    object
 2   emailAddress  0 non-null      object
dtypes: object(3)
memory usage: 2.5+ KB
